<a href="https://colab.research.google.com/github/pankajrawat9075/CS6910_Assignment_1/blob/main/Final_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import the Libraries

In [28]:
!pip install wandb -qU

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import wandb
from keras.datasets import mnist, fashion_mnist
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

In [30]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## DATA Preparation

###Load Dataset

In [31]:
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()

# Split the X_train into a training set and validation set
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.1, random_state=42)

###Summarize loaded dataset

In [32]:
print('Train: X = %s, y = %s' % (trainX.shape, trainY.shape))
print('Validation: X = %s, y = %s' % (valX.shape, valY.shape))
print('Test: X = %s, y = %s' % (testX.shape, testY.shape))

Train: X = (54000, 28, 28), y = (54000,)
Validation: X = (6000, 28, 28), y = (6000,)
Test: X = (10000, 28, 28), y = (10000,)


### Display all class_names images

In [33]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="dl_assignment_1"
)
# Get the unique labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
unique_labels = np.unique(trainY)
print("unique_labels = %s" % (unique_labels))

# Create a subplot for each label
image_list = []

# Loop through the labels and display an image for each label
for i, label in enumerate(unique_labels):
    # Get the first image with this label
    img = trainX[trainY == label][0]
    # wandb.log({class_names[label] : axes[i].inshoe(img, cmap = 'gray')})
    # Plot the image
    image_list.append(wandb.Image(img, caption=class_names[label]))
    

wandb.log({"Dataset":image_list})   


Training_Loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation_Loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training_Loss,2.34364
Training_accuracy,17.92037
Validation_Loss,2.34983
epoch,40
val_accuracy,17.18333


unique_labels = [0 1 2 3 4 5 6 7 8 9]


### Data Preprocessing

In [34]:
# Normalize the pixel values to the range [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0
valX = valX.astype('float32') / 255.0 

trainSize = trainY.shape[0]
testSize = testY.shape[0]

#One-Hot encoding for trainY and testY and valY
y_train = np.zeros(( 10, trainSize ))
y_val = np.zeros(( 10, 6000 ))
y_test = np.zeros(( 10, testSize ))

for i in range(0, trainSize ):
    y_train[trainY[i]][i] = 1

for i in range(6000):
    y_val[valY[i]][i] = 1

for i in range(0, testSize ):
    y_test[testY[i]][i] = 1

trainY = y_train
valY = y_val
testY = y_test

# reshape the X matrices
trainX = trainX.reshape(trainX.shape[0], 784)
valX = valX.reshape(6000, 784)
testX = testX.reshape(10000, 784)

trainX = trainX.T
valX = valX.T
testX = testX.T

##Neural network 

### Activation functions

In [35]:
# sigmoid function that handles overflow
def sigmoid(x):
    x = 1.0 / (1.0 + np.exp(-np.clip(x, -500, 500)))
    return x

def sigmoid_deriv(x):
    s = sigmoid(x)
    return np.multiply(s, np.subtract(1, s))

def ReLU( x):
    """
    Rectified Linear Unit (ReLU) activation function that avoids overflow.
    """
    return np.maximum(0, x)

def ReLU_deriv( x):
    """
    Derivative of the ReLU activation function that avoids overflow.
    """
    return np.where(x > 0, 1, 0)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    tanh_x = tanh(x)
    return 1 - tanh_x**2

def softmax(x):
    x -= np.max(x, axis = 0)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis = 0)

def softmax_derivative(x):
    return softmax(x) * (1-softmax(x))

def identity(x):
    return x

### Forward Propogation

In [36]:
def forward_prop(W, B, x, act_func = 'sigmoid'):

    '''Function to forward propagate a minibatch of data once through the NN

    Parameters
    ----------
    x: numpy array
        data in (features,batch_size) format

    W: numpy array
        contains all weights

    B: numpy array
        contains all biases

    act_func: string
        activation function to be used except the output layer

    Returns
    -------
    Y_: numpy array
        contains the output probabilities for each class and each data sample after 1 pass
    H: numpy array
        contains all post-activations
    A: numpy array
        contsins all pre-activations

    '''
    L = W.shape[0]+1
    A = [None] * (L - 1)

    H = [None] * (L - 1)
    Y_ = [None]

    A[0] = B[0].reshape(-1, 1) + np.matmul(W[0], x) # for the first pre-activation layer x is the input 

    for i in range(0, L-2):
        if act_func == "sigmoid":
            H[i] = sigmoid(A[i])         # computing the activation layer 

        elif act_func == "ReLU":
            H[i] = ReLU(A[i]) 

        elif act_func == "tanh":
            H[i] = tanh(A[i]) 

        A[i+1] = B[i+1].reshape(-1, 1) + np.matmul(W[i+1], H[i])

    Y_ = softmax(A[L-2])

    return A, H, Y_


### Backward Propogation

In [37]:
def back_prop(W, B,  x, y, A, H, Y_, act_func, loss_type):
    '''Function to calculate gradients for a minibatch of data once through the NN through backpropagation

    Parameters
    ----------
    Y_: numpy array
        output from forward propagation/ class probabilities

    y: numpy array
        actual class class_names in one hot
     
    H: numpy array
        post-activations

    A: numpy array
        pre-activations   

    W: numpy array
        contains all weights

    B: numpy array
        contains all biases

    act_func: string
        activation function to be used except the output layer

    batch_size: int
        mini-batch-size

    loss_type: string
        loss function (MSE/Categorical crossentropy)


    Returns
    -------
    del_W: numpy array
        gradients with respect to weights
        
    del_B: numpy array
        gradients with respect to biases

    '''
    L = W.shape[0]+1

    del_A,  del_H = A,  H  # creating gradient variables
    del_W = [None] * (L-1)
    del_B = [None] * (L-1)

    if loss_type == "cross_entropy":
        del_A[-1] = -(y - Y_)

    elif loss_type == "mean_squared_error":
        del_A[-1] = (2 / y.shape[1]) * softmax_derivative(Z[L-2])

    for i in range(L-2, 0, -1):
        del_W[i] = np.matmul(del_A[i],  np.transpose( H[i-1]))   # compute gradients with respect to weihts and bias
        del_B[i] = del_A[i]
        del_B[i] = np.array(np.sum(del_B[i], axis = 1))

        del_H[i-1] = np.matmul(np.transpose(W[i]), del_A[i])

        if act_func == "sigmoid":
            del_A[i-1] = del_H[i-1] * sigmoid_deriv(A[i-1])
        elif act_func == "ReLU":
            del_A[i-1] = del_H[i-1] * ReLU_deriv(A[i-1])
        elif act_func == "tanh":
            del_A[i-1] = del_H[i-1] * tanh_deriv(A[i-1])  

    del_W[0] = np.matmul(del_A[0] , np.transpose(x))  # compute gradients with respect to weihts and bias
    del_B[0] = del_A[0]

    del_B[0] = np.array(np.sum(del_B[0], axis = 1))


    return del_W, del_B

### Training the network

In [38]:
def train(layers, X, Y, epochs, alpha, activation_func, optimizer, batch_size, weight_init, weight_decay, loss_type, momentum = 0.9, 
          beta = 0.9, beta1 = 0.9, beta2 = 0.999, eps = 0.00001):   
    """
    parameters:

    layers: list
        contains all the layers no. of neurons

    alpha : learning rate

    return:

    W: numpy array
        contains all weights

    B: numpy array
        contains all biases

    """
    L = len(layers) # no. of layers

    # initialize the weights and biases
    W = []
    B = []
    u_b, u_w, v_w, v_b = [], [], [], []
    
    # initialize the u_b, u_w, v_w, v_b

    for i in range(1, L):
        temp_w = np.zeros((layers[i], layers[i-1]))
        temp_b = np.zeros((layers[i]))
        u_w.append(temp_w)
        v_w.append(temp_w)
        u_b.append(temp_b)
        v_b.append(temp_b)

    np.random.seed(42)

    # initialize weights and biases
    if weight_init == "random":
        for i in range(1, L):
            w = np.random.randn(layers[i], layers[i-1]) * 0.01
            b = np.random.randn(layers[i]) * 0.01 # b is bias vector
            W.append(w)
            B.append(b)

    if weight_init == "Xavier":
        for i in range(1, L):
            w = np.random.randn(layers[i], layers[i-1]) * np.sqrt(2.0 / (layers[i] + layers[i-1]))
            b = np.random.randn(layers[i]) * np.sqrt(2.0 / (layers[i] + layers[i-1]))       # b is bias vector
            W.append(w)
            B.append(b)
    
    W = np.array(W)
    B = np.array(B)
    u_w = np.array(u_w)
    v_w = np.array(v_w)
    u_b = np.array(u_b)
    v_b = np.array(v_b)
    learning_rate = alpha
    

    for epoch in range(epochs):
        alpha = learning_rate/(epoch+1)
        for i in range(0, X.shape[1], batch_size):
            batch_count = batch_size

            if i + batch_size > X.shape[1]: # the last mini-batch might contain fewer than "batch_size" examples
                batch_count = X.shape[1] - i + 1

            if optimizer == 'sgd':

                A, H, Y_ = forward_prop(W, B, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W, B, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type)

                W, B = update_parms_sgd(W, B, alpha, del_w, del_b, weight_decay)

            elif optimizer == 'momentum':

                A, H, Y_ = forward_prop(W, B, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W, B, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type) # compute the gradient at the lookahead

                u_w = momentum * u_w + del_w
                u_b = momentum * u_b + del_b
                W, B = update_parms_momentum(W, B, alpha, u_w, u_b,  weight_decay)

            elif optimizer == 'nag':

                A, H, Y_ = forward_prop(W - beta * u_w, B - alpha * u_b, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W - beta * u_w, B - alpha * u_b, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type) # compute the gradient at the lookahead

                u_w = momentum * u_w + del_w
                u_b = momentum * u_b + del_b
                
                W, B = update_parms_nag(W, B, alpha, u_w, u_b,  weight_decay)

            elif optimizer == 'rmsprop':

                A, H, Y_ = forward_prop(W, B, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W, B, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type) # compute the gradient at the lookahead

                
                W, B, u_w, u_b = update_parms_rmsprop(W, B, alpha, u_w, u_b, del_w, del_b, eps, beta,  weight_decay)

            elif optimizer == 'adam':

                A, H, Y_ = forward_prop(W, B, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W, B, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type) # compute the gradient at the lookahead

                
                W, B, u_w, u_b, v_w, v_b = update_parms_adam(W, B, alpha, u_w, u_b, v_w, v_b, del_w, del_b, eps, beta1, beta2, epoch,  weight_decay)

            elif optimizer == 'nadam':

                A, H, Y_ = forward_prop(W, B, X[:, i:i+batch_count], activation_func)
                del_w, del_b = back_prop(W, B, X[:, i:i+batch_count], Y[:, i:i+batch_count], A, H, Y_, activation_func, loss_type) # compute the gradient at the lookahead

                W, B, u_w, u_b, v_w, v_b = update_parms_nadam(W, B, alpha, u_w, u_b, v_w, v_b, del_w, del_b, eps, beta1, beta2, epoch,  weight_decay)

        show(W, B, activation_func, epoch, loss_type, weight_decay)

    return W, B


### log data to wandb

In [39]:
def show(W, B, activ_f, epoch, loss_type, wd):
    wandb.log({"epoch": epoch+1})

    if(loss_type == "cross_entropy"):

        # print the cross entropy validation loss
        val_loss = loss(valX, valY, W, B, activ_f, wd)
        wandb.log({"Validation_Loss": val_loss})

        # print the cross entropy validation accuracy
        val_accuracy = accuracy(valX, valY, W, B, activ_f)
        wandb.log({"val_accuracy": val_accuracy})

        # print the cross entropy training loss
        training_loss = loss(trainX, trainY, W, B, activ_f, wd)
        wandb.log({"Training_Loss": training_loss})

        # print the cross entropy training accuracy
        training_accuracy = accuracy(trainX, trainY,W, B, activ_f)
        wandb.log({"Training_accuracy": training_accuracy})

    elif(loss_type == "mean_squared_error"):

        # print the mean_squared_error validation loss
        val_loss = mean_squared_error(valX, valY, W, B, activ_f, wd)
        wandb.log({"Validation_Loss": val_loss})

        # print the mean_squared_error validation accuracy
        val_accuracy = accuracy(valX, valY, W, B, activ_f)
        wandb.log({"val_accuracy": val_accuracy})

        # print the mean_squared_error training loss
        training_loss = mean_squared_error(trainX, trainY, W, B, activ_f, wd)
        wandb.log({"Training_Loss": training_loss})

        # print the mean_squared_error training accuracy
        training_accuracy = accuracy(trainX, trainY,W, B, activ_f)
        wandb.log({"Training_accuracy": training_accuracy})
    


### Optimizers

In [40]:

def update_parms_sgd(W, B, alpha, del_w, del_b, wd):
    W = W - alpha * np.array(np.array(del_w)) - alpha * wd * W
    B = B - alpha * np.array(del_b) - alpha * wd * B
    return W, B

def update_parms_nag(W, B, alpha, u_w, u_b, wd):
    W = W - alpha * np.array(np.array(u_w)) - alpha * wd * W
    B = B - alpha * np.array(u_b)- alpha * wd * B
    return W, B

def update_parms_momentum(W, B, alpha, u_w, u_b, wd):
    W = W - alpha * np.array(np.array(u_w)) - alpha * wd * W
    B = B - alpha * np.array(u_b) - alpha * wd * B
    return W, B 

def update_parms_rmsprop(W, B, alpha, u_w, u_b, del_w, del_b, eps, beta, wd):

    for i in range(W.shape[0]):
        u_w[i] = beta * u_w[i] + (1-beta)*del_w[i]**2
        u_b[i] = beta * u_b[i] + (1-beta)*del_b[i]**2
        W[i] = W[i] - alpha * np.array(np.array(del_w[i])) / (np.sqrt(u_w[i]) + eps) - alpha * wd * W[i]
        B[i] = B[i] - alpha * np.array(del_b[i]) / (np.sqrt(u_b[i]) + eps) - alpha * wd * B[i]
    return W, B, u_w, u_b

def update_parms_adam(W, B, alpha, u_w, u_b, v_w, v_b, del_w, del_b, eps, beta, beta2, epoch, wd):

    for i in range(W.shape[0]):
        v_w[i] = beta * v_w[i] + (1-beta) * del_w[i]
        v_b[i] = beta * v_b[i] + (1-beta) * del_b[i]

        v_w_hat = v_w[i] / (1 - beta ** (epoch+1))
        v_b_hat = v_b[i] / (1 - beta ** (epoch+1))

        u_w[i] = beta2 * u_w[i] + (1-beta2)*del_w[i]**2
        u_b[i] = beta2 * u_b[i] + (1-beta2)*del_b[i]**2

        u_w_hat = u_w[i] / (1- beta2 ** (epoch+1))
        u_b_hat = u_b[i] / (1- beta2 ** (epoch+1))

        W[i] = W[i] - alpha * np.array(np.array(v_w_hat)) / (np.sqrt(u_w_hat) + eps) - alpha * wd * W[i]
        B[i] = B[i] - alpha * np.array(v_b_hat) / (np.sqrt(u_b_hat) + eps) - alpha * wd * B[i]

    return W, B, u_w, u_b, v_w, v_b

def update_parms_nadam(W, B, alpha, u_w, u_b, v_w, v_b, del_w, del_b, eps, beta, beta2, epoch, wd):

    for i in range(W.shape[0]):
        v_w[i] = beta * v_w[i] + (1-beta) * del_w[i]
        v_b[i] = beta * v_b[i] + (1-beta) * del_b[i]

        v_w_hat = v_w[i] / (1 - beta ** (epoch+1))
        v_b_hat = v_b[i] / (1 - beta ** (epoch+1))

        u_w[i] = beta2 * u_w[i] + (1-beta2)*del_w[i]**2
        u_b[i] = beta2 * u_b[i] + (1-beta2)*del_b[i]**2

        u_w_hat = u_w[i] / (1- beta2 ** (epoch+1))
        u_b_hat = u_b[i] / (1- beta2 ** (epoch+1))

        W[i] = W[i] - (alpha / np.sqrt(u_w_hat + eps)) * (beta * v_w_hat + (1-beta)*del_w[i]/(1-beta**(epoch+1))) - alpha * wd * W[i]
        B[i] = B[i] - (alpha / np.sqrt(u_b_hat + eps)) * (beta * v_b_hat + (1-beta)*del_b[i]/(1-beta**(epoch+1))) - alpha * wd * B[i]

    return W, B, u_w, u_b, v_w, v_b



### Predictions and Evaluations and Loss function

In [41]:
def predict(X, W, B, act_fun):
    '''
    forward propagate once and calculate class_names

    '''
    _, _, output = forward_prop(W, B, X, act_fun)
    predictions = np.argmax(output, axis=0)
    return predictions

def accuracy(X, Y, W, B, activation_f):
    test_predictions = predict(X, W, B, activation_f)
    y_test = np.argmax(Y, axis=0)
    return accuracy_score(y_test, test_predictions) * 100

def loss(X, Y, W, B, activation_f, wd):

    _, _, output = forward_prop(W, B, X, activation_f)
    output = output.T
    Y = Y.T
    eps = 1e-12
    loss = -np.mean(np.sum(Y * np.log(output + eps), axis=1))
    return loss + 0.5 * wd ** 2

# added the squared loss funtion
def mean_squared_error(X, Y, W, B, activation_f, wd):

    _, _, output = forward_prop(W, B, X, activation_f)
    output = output.T
    Y = Y.T
    loss = np.mean(np.sum((Y - output) ** 2, axis = 1))
    return loss + 0.5 * wd ** 2

def evaluate(X_train, y_train, X_test, y_test, W, B, activation_f):
    '''
    print train,test accuracies and the classification report using sklearn

    '''
    y_train = np.argmax(y_train, axis=0)
    train_predictions = predict(X_train, W, B, activation_f)
    y_test = np.argmax(y_test, axis=0)
    test_predictions = predict(X_test, W, B, activation_f)

    print("Training accuracy = ", accuracy_score(y_train, train_predictions))
    print("Test accuracy = ", accuracy_score(y_test, test_predictions))

    return train_predictions, test_predictions

## Hyperparameter tuning using Wandb

#### sweep config for wandb

In [ ]:
sweep_config = {
    'method' : 'bayes',
    'name' : 'first sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'weight_init': {
            'values': ['random', "Xavier"]
        },
        'epochs': {
            'values' : [5, 10,20, 40]
        },
        "hidden_layers": {
            "values": [ 3,4,5,6]
        },
        "size_of_layer": {
            "values": [ 32, 64,128]
        },
        'learning_rate': {
            'values': [0.001, 0.0001,0.00001]
        },
        
        'batch_size': {
            'values': [32, 64, 128]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop', 'nestrov', 'momentum']
        },
        'activation': {
            'values': ['ReLU', 'sigmoid', 'tanh']
        },
        'weight_decay':{
            'values': [0.0001, 0.0005, 0]
        }
    }
}


#### train_wand function

In [ ]:
def train_wand(config=None, loss_type = "cross_entropy"):
    config_defaults = {
        'epochs': 10,
        'batch_size': 64,
        'learning_rate': 1e-3,
        'activation_f': 'ReLU',
        'optimizer': 'adam',
        'init_mode': 'xavier',
        'L2_lamb': 0,
        'num_neurons': 64,
        'num_hidden': 3
    }
    wandb.init(config = config)
    config = wandb.config
    layers = [config.size_of_layer] * (config.hidden_layers+1)
    layers[0] = 28*28
    layers.append(10)
    name='hl_'+str(config.hidden_layers)+"_lr_"+str(config.learning_rate)+"_bs_"+str(config.batch_size)+"_opt_"+str(config.optimizer)+ '_act_'+str(config.activation)
    wandb.init(name = name)
    W, B = train(layers, trainX, trainY, epochs=config.epochs, alpha = config.learning_rate, activation_func=config.activation, 
              optimizer =config.optimizer, batch_size=config.batch_size, weight_init = config.weight_init,weight_decay = config.weight_decay, loss_type )




#### Wand Sweep

In [2]:
# Initialize WandB sweep
sweep_id = wandb.sweep(sweep_config,project="dl_assignment_1")
wandb.agent(sweep_id=sweep_id,function=train_wand)

## Testing the Best Model

In [42]:
learning_rate = 0.001
size_of_layer = 128
epochs = 40
activation_func = 'tanh'
optimizer = 'RMSprop'
batch_size = 128
weight_init = 'Xavier'
weight_decay = 0
hidden_layers = 6

In [43]:
layers = [size_of_layer] * (hidden_layers+1)
layers[0] = 28*28
layers.append(10)
W, B = train(layers, trainX, trainY, epochs=epochs, alpha = learning_rate, activation_func=activation_func, 
              optimizer =optimizer, batch_size=batch_size, weight_init = weight_init,weight_decay = weight_decay, loss_type = "cross_entropy")


In [46]:
train_predictions, test_predictions = evaluate(trainX, trainY, testX, testY, W, B, activation_func)

Training accuracy =  0.8906666666666667
Test accuracy =  0.8856


## Confusion matrix

### confustion matrix with hovering

In [ ]:
import plotly.graph_objs as go

y_pred = test_predictions
y_true = testY

# Create the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Create the hover text for the plot
hovertext = []
for i in range(len(class_names)):
    for j in range(len(class_names)):
        text = f"True class: {class_names[i]}<br>Predicted class: {class_names[j]}"
        if i == j:
            text += f"<br>Count: {cm[i][j]}"
        else:
            text += f"<br>Count: {cm[i][j]}<br>Predicted class for {class_names[i]}: {class_names[y_pred[(y_true == i) & (y_pred == j)][0]]}"
        hovertext.append(text)

# Create the plot
fig = go.Figure(data=go.Heatmap(z=cm, x=class_names, y=class_names, colorscale='Viridis', hovertext=hovertext))

# Update the plot layout
fig.update_layout(title='Confusion Matrix for Fashion MNIST', xaxis_title='Predicted Label', yaxis_title='True Label')

# Show the plot
fig.show()

###confustion matrix with accuracies

In [49]:
y = np.argmax(testY, axis = 0)
cnf = confusion_matrix(y, test_predictions, normalize='true')
plt.figure(figsize=(12, 8))
ax = sns.heatmap(cnf, annot=cnf,xtickclass_names=class_names, ytickclass_names=class_names)
ax.set_title("Confusion Matrix", size=16)
ax.set_xlabel("Predicted Class", size=14)
ax.set_ylabel("True Class", size=14)
plt.savefig("confusion matrix")
plt.show()

## Training the model on mean square error loss function

the plots for accuracies and losses has been plotted in the report and also available in the wandb report

In [ ]:
layers = [size_of_layer] * (hidden_layers+1)
layers[0] = 28*28
layers.append(10)
W, B = train(layers, trainX, trainY, epochs=epochs, alpha = learning_rate, activation_func=activation_func, 
              optimizer =optimizer, batch_size=batch_size, weight_init = weight_init,weight_decay = weight_decay, loss_type = "cross_entropy")


## Testing 3 models on MNSIT Dataset

### create dataset

In [ ]:
(trainX, trainY), (testX, testY) = mnist.load_data()

# Split the X_train into a training set and validation set
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.1, random_state=42)

### data processing

In [ ]:
# Normalize the pixel values to the range [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0
valX = valX.astype('float32') / 255.0 

trainSize = trainY.shape[0]
testSize = testY.shape[0]

#One-Hot encoding for trainY and testY and valY
y_train = np.zeros(( 10, trainSize ))
y_val = np.zeros(( 10, 6000 ))
y_test = np.zeros(( 10, testSize ))

for i in range(0, trainSize ):
    y_train[trainY[i]][i] = 1

for i in range(6000):
    y_val[valY[i]][i] = 1

for i in range(0, testSize ):
    y_test[testY[i]][i] = 1

trainY = y_train
valY = y_val
testY = y_test

# reshape the X matrices
trainX = trainX.reshape(trainX.shape[0], 784)
valX = valX.reshape(6000, 784)
testX = testX.reshape(10000, 784)

trainX = trainX.T
valX = valX.T
testX = testX.T

### Testing for Hyperparameter Config 1

In [ ]:
learning_rate = 0.001
size_of_layer = 128
epochs = 40
activation_func = 'tanh'
optimizer = 'RMSprop'
batch_size = 128
weight_init = 'Xavier'
weight_decay = 0
hidden_layers = 6

In [ ]:
layers = [size_of_layer] * (hidden_layers+1)
layers[0] = 28*28
layers.append(10)
W, B = train(layers, trainX, trainY, epochs=epochs, alpha = learning_rate, activation_func=activation_func, 
              optimizer =optimizer, batch_size=batch_size, weight_init = weight_init,weight_decay = weight_decay, loss_type = "cross_entropy")


In [50]:
train_predictions, test_predictions = evaluate(trainX, trainY, testX, testY, W, B, activation_func)

Training accuracy =  0.9992592592592593
Test accuracy =  0.9762


### Config 2

In [ ]:
learning_rate = 0.001
size_of_layer = 128
epochs = 20
activation_func = 'tanh'
optimizer = 'nadam'
batch_size = 128
weight_init = 'Xavier'
weight_decay = 0.005
hidden_layers = 6


layers = [size_of_layer] * (hidden_layers+1)
layers[0] = 28*28
layers.append(10)
W, B = train(layers, trainX, trainY, epochs=epochs, alpha = learning_rate, activation_func=activation_func, 
              optimizer =optimizer, batch_size=batch_size, weight_init = weight_init,weight_decay = weight_decay, loss_type = "cross_entropy")


In [51]:
train_predictions, test_predictions = evaluate(trainX, trainY, testX, testY, W, B, activation_func)

Training accuracy =  0.9938
Test accuracy =  0.9799


### config 3

In [ ]:
learning_rate = 0.001
size_of_layer = 64
epochs = 10
activation_func = 'tanh'
optimizer = 'adam'
batch_size = 128
weight_init = 'Xavier'
weight_decay = 0.005
hidden_layers = 5


layers = [size_of_layer] * (hidden_layers+1)
layers[0] = 28*28
layers.append(10)
W, B = train(layers, trainX, trainY, epochs=epochs, alpha = learning_rate, activation_func=activation_func, 
              optimizer =optimizer, batch_size=batch_size, weight_init = weight_init,weight_decay = weight_decay, loss_type = "cross_entropy")


In [52]:
train_predictions, test_predictions = evaluate(trainX, trainY, testX, testY, W, B, activation_func)

Training accuracy =  0.9824
Test accuracy =  0.9681
